# Agent2Agent Especializado con Router Interno (Batman)

## Objetivo
Extender el ejercicio agent2agent con una arquitectura de roles especializados:
- **RouterAgent**: clasifica la consulta en una ruta semantica.
- **SpecialistAgent** (timeline, villains, strategy, general): recupera evidencia y responde.

Si la primera respuesta sale con grounding bajo, el orquestador pide una segunda opinion al agente `general`.

## Flujo

```mermaid
flowchart TD
  A["User Query"] --> B["RouterAgent"]
  B --> C["SpecialistAgent (ruta seleccionada)"]
  C --> D["Grounding Check"]
  D -->|"bajo"| E["General Specialist (segunda opinion)"]
  D -->|"ok"| F["Final Answer"]
  E --> F
```

## Marco teorico: Especializacion por roles

### Por que especializar agentes

Cada `SpecialistAgent` en esta arquitectura tiene un **system prompt optimizado para su dominio**:
- `TimelineAgent`: prioriza orden temporal y transiciones narrativas.
- `VillainsAgent`: prioriza motivaciones, metodos y conflicto heroe-villano.
- `StrategyAgent`: prioriza decisiones tacticas y trade-offs operativos.
- `GeneralAgent`: prioriza una explicacion balanceada de hechos canonicos.

La especializacion tiene un beneficio fundamental: **un LLM con un system prompt focalizado produce respuestas mas coherentes y relevantes que uno con un prompt generico**. Esto es analogo al principio de "single responsibility" en ingenieria de software — cada agente hace una cosa bien.

Ademas, el `RouterAgent` en esta arquitectura **no es un LLM** — es un router heuristico basado en keywords. Esto tiene dos ventajas operativas:
1. **Latencia reducida**: el routing es O(n) con n = numero de keywords, negligible comparado con un forward pass de LLM.
2. **Costo cero de API**: no consume tokens de la API para decidir la ruta.

### El patron de "segunda opinion"

Cuando el agente primario produce una respuesta con grounding bajo (`< 0.2`), el orquestador consulta al agente `general` como fallback. Este patron tiene analogias en multiple dominios:

- **Mixture of Experts (MoE)**: en redes neuronales, diferentes "expertos" se especializan en subconjuntos de datos, con un gating network que decide cual activar. Aqui el router es el gate y los specialists son los expertos.
- **Sistemas de segunda opinion medica**: cuando el diagnostico del especialista es incierto, se consulta a un generalista para validar o complementar.
- **Escalamiento en soporte tecnico**: si el agente de nivel 2 no resuelve, se escala al nivel 3 (mas general pero con mas contexto).

### Threshold 0.2 vs 0.18

En la notebook anterior (Agent2Agent simple), el threshold de grounding era **0.18**. Aqui usamos **0.2**. La razon:
- Los agentes especializados deberian producir respuestas de **mayor calidad** dentro de su dominio, porque su system prompt esta optimizado para esa tarea.
- Si un agente especializado no alcanza 0.2 de grounding, es una senal mas fuerte de que algo salio mal (query fuera de dominio, documentos insuficientes, etc.).
- Un threshold ligeramente mas alto para agentes especializados actua como un **quality gate mas estricto**, lo cual es deseable cuando tienes un fallback disponible (el agente general).

In [ ]:
from __future__ import annotations

from dataclasses import dataclass
from pathlib import Path
import sys
import time
import pandas as pd

ROOT = Path.cwd()
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

from scripts.common import generate_answer
from scripts.evaluation import groundedness_score
from scripts.vector_store_lab import build_index_from_json

OUTPUTS_DIR = ROOT / 'outputs'
OUTPUTS_DIR.mkdir(parents=True, exist_ok=True)
DATA_PATH = ROOT / 'data' / 'batman_comics.json'

In [ ]:
db, chunks, index_stats, chunk_stats = build_index_from_json(
    json_path=DATA_PATH,
    persist_dir=OUTPUTS_DIR / 'chroma_agent2agent_specialized',
    collection_name='agent2agent_specialized_batman',
    chunk_size=800,
    chunk_overlap=120,
    embedding_model='text-embedding-3-small',
)

print('Index stats:', index_stats)
print('Chunk stats:', chunk_stats)

In [ ]:
ROUTE_CONFIG = {
    'timeline': {
        'focus_hint': 'Prioriza orden temporal, etapas y transiciones narrativas.',
        'system_prompt': (
            'Eres TimelineAgent. Responde como historiador tecnico del canon de Batman. '
            'Usa evidencia concreta y citas [D#].'
        ),
    },
    'villains': {
        'focus_hint': 'Prioriza motivaciones, metodos y dano psicologico de antagonistas.',
        'system_prompt': (
            'Eres VillainsAgent. Analiza conflicto heroe-villano, patrones de amenaza y riesgos. '
            'Usa solo el contexto y cita [D#].'
        ),
    },
    'strategy': {
        'focus_hint': 'Prioriza tacticas, contingencias, trade-offs y decisiones operativas.',
        'system_prompt': (
            'Eres StrategyAgent. Explica decisiones estrategicas de Batman con precision operativa. '
            'Usa evidencia y cita [D#].'
        ),
    },
    'general': {
        'focus_hint': 'Prioriza una explicacion balanceada de hechos canonicos verificables.',
        'system_prompt': (
            'Eres GeneralAgent. Sintetiza de forma rigurosa y didactica sin inventar datos. '
            'Usa contexto y cita [D#].'
        ),
    },
}

In [ ]:
@dataclass
class RouterAgent:
    def route(self, query: str) -> str:
        q = query.lower()
        if any(term in q for term in ['orden', 'cronologia', 'timeline', 'inicio', 'evolucion', 'despues']):
            return 'timeline'
        if any(term in q for term in ['joker', 'bane', 'hush', 'villano', 'enemigo', 'tribunal', 'owls']):
            return 'villains'
        if any(term in q for term in ['estrategia', 'tactica', 'plan', 'contingencia', 'liga', 'justicia']):
            return 'strategy'
        return 'general'


@dataclass
class SpecialistAgent:
    route: str
    vector_db: object
    model: str = 'gpt-5-mini'
    embedding_model: str = 'text-embedding-3-small'
    k: int = 6

    def run(self, query: str) -> dict:
        cfg = ROUTE_CONFIG[self.route]
        rewritten_query = f"{query} {cfg['focus_hint']}"
        docs, retrieval_provider = self.vector_db.query(
            query_text=rewritten_query,
            n_results=self.k,
            embedding_model=self.embedding_model,
        )
        answer, llm_provider = generate_answer(
            query=query,
            contexts=[str(doc.get('text', '')) for doc in docs],
            model=self.model,
            system_prompt=cfg['system_prompt'],
        )
        grounding = groundedness_score(answer, [doc.get('text', '') for doc in docs])
        return {
            'route': self.route,
            'answer': answer,
            'groundedness': grounding,
            'retrieved_docs': len(docs),
            'llm_provider': llm_provider,
            'retrieval_provider': retrieval_provider,
            'rewritten_query': rewritten_query,
        }


class SpecializedAgent2AgentOrchestrator:
    def __init__(self, router: RouterAgent, specialists: dict[str, SpecialistAgent], threshold: float = 0.2) -> None:
        self.router = router
        self.specialists = specialists
        self.threshold = threshold

    def run(self, query: str) -> dict:
        t0 = time.perf_counter()
        trace = []

        route = self.router.route(query)
        trace.append({'agent': 'RouterAgent', 'message': f'Ruta seleccionada: {route}'})

        primary = self.specialists[route].run(query)
        trace.append({
            'agent': f'SpecialistAgent[{route}]',
            'message': f"Grounding={primary['groundedness']}, docs={primary['retrieved_docs']}",
        })

        selected = primary
        second_opinion_used = False
        if primary['groundedness'] < self.threshold and route != 'general':
            backup = self.specialists['general'].run(query)
            trace.append({
                'agent': 'SpecialistAgent[general]',
                'message': f"Second opinion grounding={backup['groundedness']}",
            })
            if backup['groundedness'] >= primary['groundedness']:
                selected = backup
                second_opinion_used = True

        latency = round(time.perf_counter() - t0, 4)
        return {
            'query': query,
            'selected_route': route,
            'final_answer': selected['answer'],
            'final_groundedness': selected['groundedness'],
            'retrieved_docs': selected['retrieved_docs'],
            'llm_provider': selected['llm_provider'],
            'retrieval_provider': selected['retrieval_provider'],
            'second_opinion_used': second_opinion_used,
            'latency_seconds': latency,
            'trace': trace,
        }

In [ ]:
router_agent = RouterAgent()
specialists = {
    route: SpecialistAgent(route=route, vector_db=db, model='gpt-5-mini', embedding_model='text-embedding-3-small', k=6)
    for route in ROUTE_CONFIG
}
orchestrator = SpecializedAgent2AgentOrchestrator(router=router_agent, specialists=specialists, threshold=0.2)
print('Specialized agent2agent orchestrator ready.')

In [ ]:
test_query = 'Compara la evolucion tactica de Batman desde Year One hasta su enfrentamiento con Bane.'
test_result = orchestrator.run(test_query)
pd.DataFrame(test_result['trace'])

In [ ]:
print('Query:')
print(test_result['query'])
print('\nSelected route:', test_result['selected_route'])
print('Second opinion used:', test_result['second_opinion_used'])
print('Groundedness:', test_result['final_groundedness'])
print('\nAnswer:\n')
print(test_result['final_answer'])

## Ejercicio

Corre este batch y revisa como el router distribuye queries por especialista.

In [ ]:
queries = [
    'Ordena cronologicamente los hitos clave de Batman entre Year One y Dark Knight Returns.',
    'Que diferencia hay entre la metodologia de Bane y la del Joker para quebrar a Batman?',
    'Por que Batman diseña planes de contingencia contra la Liga de la Justicia?',
    'Que revela Court of Owls sobre los puntos ciegos de Bruce Wayne?',
    'Resume el rol de Robin en la evolucion del enfoque de Batman.',
]

rows = []
for q in queries:
    out = orchestrator.run(q)
    rows.append({
        'query': q,
        'selected_route': out['selected_route'],
        'final_groundedness': out['final_groundedness'],
        'retrieved_docs': out['retrieved_docs'],
        'second_opinion_used': out['second_opinion_used'],
        'latency_seconds': out['latency_seconds'],
        'llm_provider': out['llm_provider'],
        'retrieval_provider': out['retrieval_provider'],
    })

results_df = pd.DataFrame(rows)
results_df

In [ ]:
route_summary = (
    results_df.groupby('selected_route', as_index=False)
    .agg(
        queries=('query', 'count'),
        avg_groundedness=('final_groundedness', 'mean'),
        second_opinion_rate=('second_opinion_used', 'mean'),
    )
    .round(4)
)
route_summary

In [ ]:
csv_path = OUTPUTS_DIR / 'agent2agent_specialized_router_results.csv'
results_df.to_csv(csv_path, index=False)
print(f'Saved: {csv_path}')

## Cierre didactico

### Conceptos clave de esta notebook

- **Especializacion mejora calidad dentro del dominio**: un agente con un system prompt focalizado produce respuestas mas coherentes que uno generico, siempre que la query haya sido correctamente ruteada.
- **El router heuristico como gate no-LLM reduce costos y latencia**: no toda decision requiere un LLM. Para routing, un clasificador basado en reglas es suficiente cuando los dominios son bien separados.
- **El patron de segunda opinion agrega resiliencia**: en vez de fallar silenciosamente cuando el agente primario tiene bajo grounding, el sistema busca una perspectiva alternativa. Esto es especialmente util para queries que caen en fronteras entre dominios.
- **Los thresholds deben adaptarse al nivel de especializacion**: agentes especializados merecen un quality gate mas estricto (0.2) que agentes generalistas (0.18), porque se espera que sean mas precisos en su dominio.

### Recapitulacion del modulo completo

A lo largo de las 5 notebooks hemos construido una progresion pedagogica:

1. **NB01**: Diseno de base vectorial — chunking, embeddings, indexacion.
2. **NB02**: Vanilla RAG vs Agentic RAG — metricas comparativas y groundedness.
3. **NB03**: Routing entre dominios — orquestacion simple con pipelines especializados.
4. **NB04**: Agent2Agent — separacion de responsabilidades entre retriever y synthesizer.
5. **NB05**: Especializacion por roles — multiples agentes con router y segunda opinion.

Cada notebook agrega complejidad arquitectonica, pero la pregunta de ingenieria siempre es la misma: **esta complejidad adicional produce mejoras medibles para mi caso de uso?** Si no, el sistema mas simple es el correcto.